# ParallelWebSearchTool

This notebook provides a quick overview for getting started with Parallel [search tool](/docs/integrations/tools/). For detailed documentation of all ParallelWebSearchTool features and configurations head to the [API reference](https://python.langchain.com/api_reference/parallel_web/search_tool/langchain_parallel_web.search_tool.ParallelWebSearchTool.html).

The ParallelWebSearchTool provides access to Parallel's Search API, which streamlines the traditional search → scrape → extract pipeline into a single API call, returning structured, LLM-optimized results.

## Overview

### Integration details

| Class | Package | Serializable | JS support |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [ParallelWebSearchTool](https://python.langchain.com/api_reference/parallel_web/search_tool/langchain_parallel_web.search_tool.ParallelWebSearchTool.html) | [langchain-parallel-web](https://python.langchain.com/api_reference/parallel_web/) | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-parallel-web?style=flat-square&label=%20) |

### Tool features

- **Real-time web search**: Access current information from the web
- **Structured results**: Returns compressed, LLM-optimized excerpts
- **Flexible input**: Support for natural language objectives or specific search queries
- **Domain filtering**: Include or exclude specific domains with source policy
- **Customizable output**: Control number of results (1-40) and excerpt length (min 100 chars)
- **Rich metadata**: Optional search timing, result counts, and query information
- **Async support**: Full async/await support with proper executor handling
- **Error handling**: Comprehensive error handling with detailed error messages

## Setup

The integration lives in the `langchain-parallel-web` package.

In [ ]:
%pip install --quiet -U langchain-parallel-web

### Credentials

Head to [Parallel](https://beta.parallel.ai) to sign up and generate an API key. Once you've done this set the PARALLEL_API_KEY environment variable:

In [ ]:
import getpass
import os

if not os.environ.get("PARALLEL_API_KEY"):
    os.environ["PARALLEL_API_KEY"] = getpass.getpass("Parallel API key:\n")

## Instantiation

Here we show how to instantiate an instance of the ParallelWebSearchTool. The tool can be configured with API key and base URL parameters:

In [ ]:
from langchain_parallel_web import ParallelWebSearchTool

# Basic instantiation - API key from environment
tool = ParallelWebSearchTool()

# With explicit API key and custom base URL
tool = ParallelWebSearchTool(
    api_key="your-api-key",
    base_url="https://api.parallel.ai",  # default value
)

## Invocation

### [Invoke directly with args](/docs/concepts/tools/#use-the-tool-directly)

You can invoke the tool with either an `objective` (natural language description) or specific `search_queries`. The tool supports various configuration options including domain filtering and metadata collection:

In [ ]:
# Using specific search queries with advanced options
result = tool.invoke(
    {
        "search_queries": [
            "AI breakthroughs 2024",
            "machine learning advances",
            "generative AI news",
        ],
        "max_results": 8,
        "max_chars_per_result": 2000,
        "source_policy": {
            "include_domains": ["arxiv.org", "nature.com"],
            "exclude_domains": ["reddit.com", "twitter.com"],
        },
        "include_metadata": True,
        "timeout": 120,  # Custom timeout in seconds
    }
)

print(result)

In [ ]:
# Using an objective (natural language) with metadata
result = tool.invoke(
    {
        "objective": "What are the latest developments in artificial intelligence in 2024?",
        "max_results": 5,
        "include_metadata": True,  # Include search timing and statistics
    }
)

print(result)

# Example response structure:
# {
#     "search_id": "search_abc123...",
#     "results": [
#         {
#             "url": "https://example.com/ai-news",
#             "title": "Latest AI Developments 2024",
#             "excerpts": [
#                 "Recent breakthrough in transformer architectures...",
#                 "New applications in computer vision..."
#             ]
#         }
#     ],
#     "search_metadata": {
#         "search_duration_seconds": 4.123,
#         "search_timestamp": "2024-01-15T10:30:00",
#         "max_results_requested": 5,
#         "actual_results_returned": 4,
#         "search_id": "search_abc123...",
#         "query_count": 1,
#         "source_policy_applied": false
#     }
# }

### [Invoke with ToolCall](/docs/concepts/tool_calling/#tool-execution)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [ ]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {
        "objective": "Find recent news about climate change initiatives",
        "max_results": 3,
        "source_policy": {"include_domains": ["ipcc.ch", "unfccc.int", "nature.com"]},
        "include_metadata": True,
    },
    "id": "call_123",
    "name": tool.name,  # "parallel_web_search"
    "type": "tool_call",
}

result = tool.invoke(model_generated_tool_call)
print(result)
print(f"Tool name: {tool.name}")  # parallel_web_search
print(f"Tool description: {tool.description}")

### [Async Usage](/docs/concepts/tools/#async)

The tool supports full async/await operations for better performance in async applications:

In [ ]:
async def search_async():
    return await tool.ainvoke(
        {
            "objective": "Latest quantum computing breakthroughs",
            "max_results": 5,
            "include_metadata": True,
        }
    )


# Run async search
result = await search_async()
print(result)

### Parameter Details and Validation

The tool performs comprehensive input validation and supports the following parameters:

#### Required Parameters (at least one):
- `objective`: Natural language description (max 5000 characters)
- `search_queries`: List of search queries (max 5 queries, 200 chars each)

#### Optional Parameters:
- `max_results`: Number of results to return (1-40, default: 10)
- `max_chars_per_result`: Characters per result excerpt (min 100, default: 1500)
- `source_policy`: Domain filtering with `include_domains` and/or `exclude_domains` lists
- `include_metadata`: Include search timing and statistics (default: True)
- `timeout`: Request timeout in seconds (optional)

#### Error Handling:
The tool provides detailed error messages for validation failures and API errors.

In [ ]:
# Example of comprehensive parameter usage
result = tool.invoke(
    {
        "objective": "Find comprehensive information about renewable energy policies in European countries",
        "max_results": 15,
        "max_chars_per_result": 2500,  # Longer excerpts for detailed information
        "source_policy": {
            "include_domains": ["europa.eu", "iea.org", "irena.org"],
            "exclude_domains": ["wikipedia.org", "reddit.com"],
        },
        "include_metadata": True,
        "timeout": 180,  # Extended timeout for comprehensive searches
    }
)

# Access results and metadata
print(f"Found {len(result['results'])} results")
if "search_metadata" in result:
    metadata = result["search_metadata"]
    print(f"Search took {metadata['search_duration_seconds']}s")
    print(f"Source policy applied: {metadata.get('source_policy_applied', False)}")

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [16]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("What are the latest breakthrough discoveries in quantum computing?")

## Best Practices

- **Use specific objectives**: More specific objectives lead to better, more targeted results
- **Apply domain filtering**: Use `source_policy` to focus on authoritative sources or exclude unreliable domains
- **Include metadata**: Set `include_metadata: True` for debugging and performance optimization
- **Handle errors gracefully**: The tool provides detailed error messages for validation and API failures
- **Use async for performance**: Use `ainvoke()` in async applications for better performance

## Response Format

The tool returns a structured dictionary with the following format:

```python
{
    "search_id": "search_abc123...",  # Unique search identifier
    "results": [  # List of search results
        {
            "url": "https://example.com/page",
            "title": "Page Title",
            "excerpts": [  # Relevant text excerpts
                "First relevant excerpt...",
                "Second relevant excerpt..."
            ]
        }
    ],
    "search_metadata": {  # Optional metadata (if include_metadata=True)
        "search_duration_seconds": 4.123,
        "search_timestamp": "2024-01-15T10:30:00",
        "max_results_requested": 10,
        "actual_results_returned": 8,
        "search_id": "search_abc123...",
        "query_count": 3,  # Number of queries used
        "queries_used": ["query1", "query2", "query3"],  # If search_queries provided
        "source_policy_applied": true,  # If source_policy was used
        "included_domains": ["nature.com"],  # Domains that were included
        "excluded_domains": ["reddit.com"]   # Domains that were excluded
    }
}
```

## API reference

For detailed documentation of all ParallelWebSearchTool features and configurations head to the API reference: https://python.langchain.com/api_reference/parallel_web/search_tool/langchain_parallel_web.search_tool.ParallelWebSearchTool.html